<a href="https://colab.research.google.com/github/VanillaVery/portfolio/blob/main/bert_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
''' v3 -> MAX_LENGTH = 200 -> gpu lost error
    v3 -> MAX_LENGTH = 150 '''

import os
import re
import numpy as np
from tqdm import tqdm
import time 
import tensorflow as tf
from transformers import *

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import pandas as pd
import matplotlib.pyplot as plt

tf.random.set_seed(1234)
np.random.seed(1234)

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attenti

In [4]:

start = time.time()
BATCH_SIZE = 32
NUM_EPOCHS = 8
VALID_SPLIT = 0.2
MAX_LEN = 200 # EDA에서 추출된 Max Length
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = "./data_out/BERT"
# os.chdir(os.path.dirname(__file__))
print(os.getcwd())
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)


loading file vocab.txt from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading configuration file config.json from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,


/content


In [5]:

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, 'train.csv')

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0)
train_data = train_data.dropna()
train_data.head()

,document,label
0,기대를 많이 해서 그런지.. 샴푸만 하고 헹궜을땐 전 좀 뻣뻣하네요. 두피세정력도 ...,1
1,길이가 생각보다 짧아요 다른색상 가지고있는데 같은 모델인데 왜 그럴까 느낌탓인가 ㅠ...,0
2,성남이 문제가 많구나,0
3,인공향신료냄새가 어마무시합니다 양념은 매운맛 이외의 맛은 잘안느껴지구요 배송은빠른편...,0
4,빅하우스 형님... 힘내 코로나걸리지마ㅠㅠ,0


In [6]:

saved_model_path = './JD_bert'

def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True   # Construct attn. masks.  
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

for train_sent, train_label in tqdm(zip(train_data["document"], train_data["label"]), total=len(train_data)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

  0%|          | 0/471188 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 471188/471188 [02:30<00:00, 3134.72it/s]

In [7]:

train_movie_input_ids = np.array(input_ids, dtype=int)
train_movie_attention_masks = np.array(attention_masks, dtype=int)
train_movie_type_ids = np.array(token_type_ids, dtype=int)
train_movie_inputs = (train_movie_input_ids, train_movie_attention_masks, train_movie_type_ids)

train_data_labels = np.asarray(train_data_labels, dtype=np.int32) #레이블 토크나이징 리스트

#Max length = 39
input_id = train_movie_input_ids[1]
attention_mask = train_movie_attention_masks[1]
token_type_id = train_movie_type_ids[1]

print(input_id)
print(attention_mask)
print(token_type_id)
print(tokenizer.decode(input_id))
print("# sents: {}, # labels: {}".format(len(train_movie_input_ids), len(train_data_labels)))

input_id = train_movie_input_ids[1]
attention_mask = train_movie_attention_masks[1]
token_type_id = train_movie_type_ids[1] 

print(input_id)
print(attention_mask)
print(token_type_id)
print(tokenizer.decode(input_id))


[   101   8934  57362   9420  66540  80001   9717  16985  48549  19709
  41442  14871  44270  84177  28911  18589   9283 118791  12030  28911
   9596   8924 118865 118671   9041 118713 119339  12030  11287    100
    119    119    102      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      

In [8]:

class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=2)

for layer in cls_model.bert.layers:
    layer.trainable=False
    for w in layer.weights: 
        w._trainable=False

optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# model_name = "tf2_bert_md"
ckpt_name = "bert_ckpt/v203"
# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=2)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)\

checkpoint_path = os.path.join(ckpt_name, 'bert_tensor-{epoch:04d}.ckpt')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)


loading configuration file config.json from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading weights file tf_model.h5 from cache at bert_ckpt/mo

bert_ckpt/v203 -- Folder already exists 



In [9]:
#십만개로 줄여보자 
# train_movie_inputs[0][0]
# train_movie_inputs[1][0]
# train_movie_inputs[2][0]
# train_data_labels[0]


import random
random.seed(605)
k=random.sample(range(len(train_movie_inputs[1])),100000)

train_movie_inputs2 = (train_movie_inputs[0][k],train_movie_inputs[1][k],train_movie_inputs[2][k])
train_data_labels2 = train_data_labels[k]


In [10]:

history = cls_model.fit(train_movie_inputs2, train_data_labels2, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])


Epoch 1/8
2500/2500 [==============================] - ETA: 0s - loss: 0.6715 - accuracy: 0.5873
Epoch 1: val_accuracy improved from -inf to 0.66610, saving model to bert_ckpt/v203/bert_tensor-0001.ckpt
2500/2500 [==============================] - 1497s 594ms/step - loss: 0.6715 - accuracy: 0.5873 - val_loss: 0.6518 - val_accuracy: 0.6661
Epoch 2/8
2500/2500 [==============================] - ETA: 0s - loss: 0.6479 - accuracy: 0.6368
Epoch 2: val_accuracy improved from 0.66610 to 0.67765, saving model to bert_ckpt/v203/bert_tensor-0002.ckpt
2500/2500 [==============================] - 1541s 617ms/step - loss: 0.6479 - accuracy: 0.6368 - val_loss: 0.6301 - val_accuracy: 0.6776
Epoch 3/8
2500/2500 [==============================] - ETA: 0s - loss: 0.6338 - accuracy: 0.6592
Epoch 3: val_accuracy improved from 0.67765 to 0.70225, saving model to bert_ckpt/v203/bert_tensor-0003.ckpt
2500/2500 [==============================] - 1500s 600ms/step - loss: 0.6338 - accuracy: 0.6592 - val_loss: 0

In [ ]:
cls_model.save('data100000_acc_70.h5')